<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/NNWND.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
device_name = tf.test.gpu_device_name()
print(device_name)
df_train = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/train_clean.csv',names=['Tweet','Emotion'])
df_test = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/test_clean.csv',names=['Tweet','Emotion'])
df_train = df.sample(frac=1)
df_test = df.sample(frac=1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/device:GPU:0


In [31]:
df_train.head(1000)

,Tweet,Emotion
9459,allergi suck my eye r even mor red than they w...,0
148,I am at work,1
5232,sometim i wish i had my own prototyp mentor,1
5611,saynerd omgg hhaha noo drag me to hell is evvi...,0
8725,calamitylil can you send out lettersemail to c...,1
...,...,...
11362,miqotu oh no that doe not sound good better ge...,1
726,i cannot sleep thi is so darn frustrat,2
5726,kmon3i lol well these girl did not and my girl...,2
6041,well here i go off to work,0


In [32]:
df_test.Emotion.unique()

array(['2', '0', '1', '3', 'emotion'], dtype=object)

In [27]:
df_train.nunique()

Tweet      12370
Emotion        5
dtype: int64

In [42]:
category_dict = {'0':[1,0,0,0],'1':[0,1,0,0],'2':[0,0,1,0],'3':[0,0,0,1],'emotion':[1,0,0,0]}
train_data_tweet = [x.lower().split() for x in df_train['Tweet']]
train_data_cat = np.array([category_dict[x] for x in df_train['Emotion']])
test_data_tweet = [x.lower().split() for x in df_test['Tweet']]
test_data_cat = np.array([category_dict[x] for x in df_test['Emotion']])

data_tweet = train_data_tweet + test_data_tweet

In [35]:
print(data_tweet[:5])
print(data_cat[:5])

[['allergi', 'suck', 'my', 'eye', 'r', 'even', 'mor', 'red', 'than', 'they', 'were', 'b4', 'i', 'think', 'ill', 'stay', 'insid', '2day'], ['i', 'am', 'at', 'work'], ['sometim', 'i', 'wish', 'i', 'had', 'my', 'own', 'prototyp', 'mentor'], ['saynerd', 'omgg', 'hhaha', 'noo', 'drag', 'me', 'to', 'hell', 'is', 'evviil', 'and', 'angel', 'and', 'demon', 'supposedli', 'suck', 'ohh', 'no', 'haha'], ['calamitylil', 'can', 'you', 'send', 'out', 'lettersemail', 'to', 'client', 'to', 'remind', 'them', 'your', 'there']]
[[1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 0 0]]


In [36]:
# Parameters
Min_count = 0
Embedding_size = 100
Window_size = 5
Negative_sampling = 00

In [43]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)

(1303276, 1724630)

In [44]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.100d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [45]:
batch_size = 100
Max_input_size = max([len(x) for x in data_tweet])


In [46]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      temp = np.array([np.array([w2v.wv.get_vector(i) for i in x if i in vocab]) for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]])
      temp = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in temp])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size=100):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      temp = np.array([np.array([vocab[i] for i in x if i in vocab.keys()]) for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]])
      temp = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in temp])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)


In [48]:
X_train = train_data_tweet
X_test = test_data_tweet
y_train = train_data_cat
y_test = test_data_cat 
result_table = []

In [49]:
epochs = 20

In [50]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("SG_LSTM")


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
124/124 [==============================] - 6s 49ms/step - loss: 1.0481 - accuracy: 0.5820
Epoch 2/20
124/124 [==============================] - 6s 50ms/step - loss: 0.9874 - accuracy: 0.5852
Epoch 3/20
124/124 [==============================] - 6s 49ms/step - loss: 0.9873 - accuracy: 0.5852
Epoch 4/20
124/124 [==============================] - 6s 48ms/step - loss: 0.9872 - accuracy: 0.5852
Epoch 5/20
124/124 [==============================] - 6s 49ms/step - loss: 0.9871 - accuracy: 0.5852
Epoch 6/20
124/124 [==============================] - 6s 49ms/step - loss: 0.9870 - accuracy: 0.5852
Epoch 7/20
124/124 [==============================] - 6s 51ms/step - loss: 0.9869 - accuracy: 0.5852
Epoch 8/20
124/124 [==============================] - 6s 49ms/step - loss: 0.9868 - accuracy: 0.5852
Epoch 9/20
124/124 [==============================] - 6s 48ms/step - loss: 0.9864 - accuracy: 0.5852
Epoch 10/20
124

In [ ]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_bi.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("SG_BI")

Epoch 1/20
49/49 [==============================] - 3s 56ms/step - loss: 1.3772 - accuracy: 0.3171
Epoch 2/20
49/49 [==============================] - 3s 57ms/step - loss: 1.3744 - accuracy: 0.3182
Epoch 3/20
49/49 [==============================] - 3s 57ms/step - loss: 1.3741 - accuracy: 0.3182
Epoch 4/20
49/49 [==============================] - 3s 59ms/step - loss: 1.3739 - accuracy: 0.3182
Epoch 5/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3736 - accuracy: 0.3182
Epoch 6/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3732 - accuracy: 0.3182
Epoch 7/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3724 - accuracy: 0.3182
Epoch 8/20
49/49 [==============================] - 3s 57ms/step - loss: 1.3717 - accuracy: 0.3190
Epoch 9/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3720 - accuracy: 0.3233
Epoch 10/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3708 - accuracy: 0.3202
Epoch 11/

In [ ]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_rnn.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("SG_RNN")

Epoch 1/20
49/49 [==============================] - 4s 78ms/step - loss: 1.3781 - accuracy: 0.3061
Epoch 2/20
49/49 [==============================] - 4s 79ms/step - loss: 1.3705 - accuracy: 0.3165
Epoch 3/20
49/49 [==============================] - 4s 79ms/step - loss: 1.3691 - accuracy: 0.3235
Epoch 4/20
49/49 [==============================] - 4s 78ms/step - loss: 1.3728 - accuracy: 0.3139
Epoch 5/20
49/49 [==============================] - 4s 78ms/step - loss: 1.3621 - accuracy: 0.3282
Epoch 6/20
49/49 [==============================] - 4s 78ms/step - loss: 1.3610 - accuracy: 0.3310
Epoch 7/20
49/49 [==============================] - 4s 79ms/step - loss: 1.3612 - accuracy: 0.3261
Epoch 8/20
49/49 [==============================] - 4s 80ms/step - loss: 1.3566 - accuracy: 0.3363
Epoch 9/20
49/49 [==============================] - 4s 79ms/step - loss: 1.3470 - accuracy: 0.3439
Epoch 10/20
49/49 [==============================] - 4s 81ms/step - loss: 1.3497 - accuracy: 0.3388
Epoch 11/

In [ ]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("CBOW_LSTM")

Epoch 1/20
49/49 [==============================] - 3s 55ms/step - loss: 1.3775 - accuracy: 0.3149
Epoch 2/20
49/49 [==============================] - 3s 54ms/step - loss: 1.3747 - accuracy: 0.3182
Epoch 3/20
49/49 [==============================] - 3s 56ms/step - loss: 1.3746 - accuracy: 0.3182
Epoch 4/20
49/49 [==============================] - 3s 56ms/step - loss: 1.3746 - accuracy: 0.3182
Epoch 5/20
49/49 [==============================] - 3s 55ms/step - loss: 1.3745 - accuracy: 0.3182
Epoch 6/20
49/49 [==============================] - 3s 61ms/step - loss: 1.3745 - accuracy: 0.3182
Epoch 7/20
49/49 [==============================] - 3s 60ms/step - loss: 1.3745 - accuracy: 0.3182
Epoch 8/20
49/49 [==============================] - 3s 62ms/step - loss: 1.3745 - accuracy: 0.3182
Epoch 9/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3744 - accuracy: 0.3182
Epoch 10/20
49/49 [==============================] - 3s 56ms/step - loss: 1.3744 - accuracy: 0.3182
Epoch 11/

In [ ]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_bi.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("CBOW_BI")

Epoch 1/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3770 - accuracy: 0.3167
Epoch 2/20
49/49 [==============================] - 3s 58ms/step - loss: 1.3745 - accuracy: 0.3182
Epoch 3/20
49/49 [==============================] - 3s 56ms/step - loss: 1.3742 - accuracy: 0.3182
Epoch 4/20
49/49 [==============================] - 3s 57ms/step - loss: 1.3740 - accuracy: 0.3182
Epoch 5/20
49/49 [==============================] - 3s 56ms/step - loss: 1.3737 - accuracy: 0.3182
Epoch 6/20
49/49 [==============================] - 3s 56ms/step - loss: 1.3734 - accuracy: 0.3182
Epoch 7/20
49/49 [==============================] - 3s 56ms/step - loss: 1.3730 - accuracy: 0.3182
Epoch 8/20
49/49 [==============================] - 3s 57ms/step - loss: 1.3725 - accuracy: 0.3182
Epoch 9/20
49/49 [==============================] - 3s 57ms/step - loss: 1.3718 - accuracy: 0.3182
Epoch 10/20
49/49 [==============================] - 3s 56ms/step - loss: 1.3696 - accuracy: 0.3192
Epoch 11/

In [ ]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_rnn.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("CBOW_RNN")

Epoch 1/20
49/49 [==============================] - 4s 80ms/step - loss: 1.3778 - accuracy: 0.3076
Epoch 2/20
49/49 [==============================] - 4s 80ms/step - loss: 1.3743 - accuracy: 0.3178
Epoch 3/20
49/49 [==============================] - 4s 78ms/step - loss: 1.3717 - accuracy: 0.3135
Epoch 4/20
49/49 [==============================] - 4s 77ms/step - loss: 1.3695 - accuracy: 0.3208
Epoch 5/20
49/49 [==============================] - 4s 78ms/step - loss: 1.3615 - accuracy: 0.3320
Epoch 6/20
49/49 [==============================] - 4s 77ms/step - loss: 1.3633 - accuracy: 0.3276
Epoch 7/20
49/49 [==============================] - 4s 79ms/step - loss: 1.3624 - accuracy: 0.3306
Epoch 8/20
49/49 [==============================] - 4s 78ms/step - loss: 1.3622 - accuracy: 0.3331
Epoch 9/20
49/49 [==============================] - 4s 78ms/step - loss: 1.3554 - accuracy: 0.3324
Epoch 10/20
49/49 [==============================] - 4s 77ms/step - loss: 1.3595 - accuracy: 0.3290
Epoch 11/

In [ ]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size=100),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size=100),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("GLOVE_LSTM")


Epoch 1/20
49/49 [==============================] - 0s 10ms/step - loss: 1.3770 - accuracy: 0.3143
Epoch 2/20
49/49 [==============================] - 0s 10ms/step - loss: 1.2938 - accuracy: 0.3845
Epoch 3/20
49/49 [==============================] - 0s 10ms/step - loss: 1.2132 - accuracy: 0.4380
Epoch 4/20
49/49 [==============================] - 0s 10ms/step - loss: 1.1619 - accuracy: 0.4561
Epoch 5/20
49/49 [==============================] - 0s 10ms/step - loss: 1.1139 - accuracy: 0.4782
Epoch 6/20
49/49 [==============================] - 0s 10ms/step - loss: 1.0609 - accuracy: 0.5145
Epoch 7/20
49/49 [==============================] - 0s 10ms/step - loss: 1.0115 - accuracy: 0.5476
Epoch 8/20
49/49 [==============================] - 0s 10ms/step - loss: 0.9671 - accuracy: 0.5804
Epoch 9/20
49/49 [==============================] - 0s 10ms/step - loss: 0.9466 - accuracy: 0.5912
Epoch 10/20
49/49 [==============================] - 0s 9ms/step - loss: 0.9082 - accuracy: 0.6137
Epoch 11/2

In [ ]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_bi.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("GLOVE_BI")

Epoch 1/20
49/49 [==============================] - 1s 11ms/step - loss: 1.3530 - accuracy: 0.3404
Epoch 2/20
49/49 [==============================] - 1s 11ms/step - loss: 1.2053 - accuracy: 0.4639
Epoch 3/20
49/49 [==============================] - 1s 11ms/step - loss: 1.0716 - accuracy: 0.5539
Epoch 4/20
49/49 [==============================] - 1s 11ms/step - loss: 0.9544 - accuracy: 0.6171
Epoch 5/20
49/49 [==============================] - 1s 11ms/step - loss: 0.8529 - accuracy: 0.6686
Epoch 6/20
49/49 [==============================] - 1s 11ms/step - loss: 0.7682 - accuracy: 0.7106
Epoch 7/20
49/49 [==============================] - 1s 11ms/step - loss: 0.6925 - accuracy: 0.7457
Epoch 8/20
49/49 [==============================] - 1s 11ms/step - loss: 0.6229 - accuracy: 0.7720
Epoch 9/20
49/49 [==============================] - 1s 12ms/step - loss: 0.5609 - accuracy: 0.7971
Epoch 10/20
49/49 [==============================] - 1s 12ms/step - loss: 0.5105 - accuracy: 0.8180
Epoch 11/

In [ ]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_rnn.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("GLOVE_RNN")

Epoch 1/20
49/49 [==============================] - 2s 40ms/step - loss: 1.3808 - accuracy: 0.3110
Epoch 2/20
49/49 [==============================] - 2s 41ms/step - loss: 1.3685 - accuracy: 0.3273
Epoch 3/20
49/49 [==============================] - 2s 40ms/step - loss: 1.3191 - accuracy: 0.3822
Epoch 4/20
49/49 [==============================] - 2s 40ms/step - loss: 1.2803 - accuracy: 0.4098
Epoch 5/20
49/49 [==============================] - 2s 40ms/step - loss: 1.2355 - accuracy: 0.4396
Epoch 6/20
49/49 [==============================] - 2s 40ms/step - loss: 1.2103 - accuracy: 0.4551
Epoch 7/20
49/49 [==============================] - 2s 40ms/step - loss: 1.1914 - accuracy: 0.4671
Epoch 8/20
49/49 [==============================] - 2s 39ms/step - loss: 1.2035 - accuracy: 0.4529
Epoch 9/20
49/49 [==============================] - 2s 40ms/step - loss: 1.1786 - accuracy: 0.4641
Epoch 10/20
49/49 [==============================] - 2s 38ms/step - loss: 1.1322 - accuracy: 0.4924
Epoch 11/

In [ ]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])

	Lstm			BiLstm			Rnn
sg [0.31857141852378845, 0.3266666531562805, 0.31761905550956726]
cbow [0.31857141852378845, 0.3261904716491699, 0.30619049072265625]
glove [0.6480952501296997, 0.7176190614700317, [0.3685714304447174]]
